In [105]:
suppressMessages(library(zellkonverter))
suppressMessages(library(SingleCellExperiment))
suppressMessages(library(miloR))
suppressMessages(library(ArchR))
library(ggplot2)
library(ggbeeswarm)
library(colorspace)
library(scater)
library(dplyr)
library(patchwork)

ERROR: Error in library(scater): there is no package called ‘scater’


In [17]:
proj <- loadArchRProject("./ArchRProject", showLogo = FALSE)

Successfully loaded ArchRProject!



In [18]:
peakMatrix <- getMatrixFromProject(proj, useMatrix = "PeakMatrix")

ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-19630f5690c538-Date-2025-03-19_Time-10-26-42.389639.log
If there is an issue, please report to github with logFile!

2025-03-19 10:32:17.348426 : Organizing colData, 5.583 mins elapsed.

2025-03-19 10:32:17.79273 : Organizing rowData, 5.59 mins elapsed.

2025-03-19 10:32:17.809588 : Organizing rowRanges, 5.59 mins elapsed.

2025-03-19 10:32:17.831129 : Organizing Assays (1 of 1), 5.591 mins elapsed.

2025-03-19 10:33:00.767198 : Constructing SummarizedExperiment, 6.306 mins elapsed.

2025-03-19 10:34:22.933205 : Finished Matrix Creation, 7.676 mins elapsed.



In [ ]:
cellColData <- proj@cellColData

In [ ]:
# PeakMatrix 与 cellcOlData列名内容不一致或顺序不一致都会报错
peak_colnames <- colnames(assay(peakMatrix))

# 按peakMatrix列名顺序重排cellColData
cellColData <- cellColData[peak_colnames, ]

In [67]:
# SingleCellExperiment对象为miloR的输入格式
sce <- SingleCellExperiment(assays = list(counts = assay(peakMatrix)), colData = cellColData)

In [68]:
# 提取LSI降维结果
lsi <- proj@reducedDims$IterativeLSI

In [69]:
# 添加结果
reducedDim(sce, "LSI") <- lsi$matSVD

Warning message in .check_reddim_names(x, value, withDimnames):
“non-NULL 'rownames(value)' should be the same as 'colnames(x)' for
'reducedDim<-'. This will be an error in the next release of
Bioconductor.”


In [98]:
unique(sce$majorType)

[1] "Epithelial"   "Myeloid"      "Endothelial"  "Mast"         "PeriVascular"
[6] "Fibroblast"   "T_cell"       "B_cell"       "Plasma"

In [70]:
# 创建 Milo 对象
milo <- Milo(sce)

In [71]:
# 构建 k 近邻图
milo <- buildGraph(milo, k = 20, d = 30, reduced.dim = "LSI")

Constructing kNN graph with k:20



In [72]:
# 定义邻域
milo <- makeNhoods(milo, prop = 0.1, k = 20, d = 30, refined = TRUE, reduced_dims = "LSI")

Checking valid object



Running refined sampling with reduced_dim



In [73]:
# 计算邻域中的细胞数（替换 "samples" 为元数据中的样本列名）
milo <- countCells(milo, meta.data = colData(sce), samples = "Sample")

Checking meta.data validity

Counting cells in neighbourhoods



In [95]:
unique(colData(sce)[, c("Sample", "neoadjuvant")])

DataFrame with 11 rows and 2 columns
                       Sample         neoadjuvant
                        <Rle>               <Rle>
10T#TTGCGAAGTCTCTAAG-1    10T without_neoadjuvant
10N#TGTGGCGAGTTCGCGC-1    10N without_neoadjuvant
13T#GTGATCAAGTGTCCCG-1    13T   after_neoadjuvant
20T#TGCTCACTCTCTATCA-1    20T without_neoadjuvant
21T#TCACCTGCAACTGCGC-1    21T without_neoadjuvant
18T#CAATCCCAGAACTCCT-1    18T without_neoadjuvant
11T#CGCACAGTCGATGTAC-1    11T   after_neoadjuvant
22T#AAAGGATAGCGTCAAG-1    22T without_neoadjuvant
12N#AGACAAAAGCGTTGCC-1    12N   after_neoadjuvant
19T#TGTAGCAGTAAGGTCG-1    19T   after_neoadjuvant
12T#GAGGTCCGTATCTGCA-1    12T   after_neoadjuvant

In [74]:
sample_metadata <- unique(colData(sce)[, c("Sample", "neoadjuvant")])
rownames(sample_metadata) <- sample_metadata$Sample

In [58]:
sample_metadata

DataFrame with 11 rows and 2 columns
    Sample         neoadjuvant
     <Rle>               <Rle>
10T    10T without_neoadjuvant
10N    10N without_neoadjuvant
13T    13T   after_neoadjuvant
20T    20T without_neoadjuvant
21T    21T without_neoadjuvant
18T    18T without_neoadjuvant
11T    11T   after_neoadjuvant
22T    22T without_neoadjuvant
12N    12N   after_neoadjuvant
19T    19T   after_neoadjuvant
12T    12T   after_neoadjuvant

In [75]:
colnames(nhoodCounts(milo))
stopifnot(all(colnames(nhoodCounts(milo)) %in% rownames(sample_metadata)))

[1] "10T" "10N" "13T" "20T" "21T" "18T" "11T" "22T" "12N" "19T" "12T"

In [79]:
# 进行差异丰度分析（替换 "condition" 为元数据中的条件列名）
DA_results <- testNhoods(milo, design = ~ neoadjuvant, design.df = sample_metadata, reduced.dim = "LSI")

Using TMM normalisation

Performing spatial FDR correction with k-distance weighting



In [92]:
unique(DA_results[["majorType"]])

[1] "T_cell"       "PeriVascular" "Myeloid"      "Fibroblast"   "Epithelial"  
[6] "Endothelial"

In [80]:
milo <- buildNhoodGraph(milo)

In [101]:
unique(milo$majorType)

[1] "Epithelial"   "Myeloid"      "Endothelial"  "Mast"         "PeriVascular"
[6] "Fibroblast"   "T_cell"       "B_cell"       "Plasma"

In [81]:
DA_results <- annotateNhoods(milo, DA_results, coldata_col = "majorType")

Converting majorType to factor...



In [103]:
unique(DA_results[['majorType']])

[1] "T_cell"       "PeriVascular" "Myeloid"      "Fibroblast"   "Epithelial"  
[6] "Endothelial"

In [89]:
# 可视化结果
DA_results <- DA_results[order(DA_results[['majorType']], decreasing=T), ]
p <- ggplot(DA_results, aes(x = logFC, y = majorType)) +
  geom_quasirandom(data=DA_results[DA_results$SpatialFDR > 0.1,], alpha=0.5, colour='grey50', size=1) + 
  geom_quasirandom(data=DA_results[DA_results$SpatialFDR <= 0.1,], aes(colour=logFC, alpha=abs(logFC)), size=1.5) + 
  geom_violin(color = "#44403f", alpha=0) +
  theme_bw() +
  scale_colour_gradient2(low=darken('blue'), mid='grey80', high=darken('#ff0000'), midpoint=0) +
  labs(x="Log Fold Change", y="majorType") +
  theme(axis.text.y=element_text(size=12,colour='black'), axis.title.y=element_text(size=14),
        axis.text.x=element_text(size=12,colour='black'), axis.title.x=element_text(size=14),
        panel.grid.major = element_blank(), 
        panel.grid.minor = element_blank()) +
  stat_summary(aes(label = sprintf("%.2f",..x..)),
        fun = mean, geom = "text") +
  guides(colour=FALSE, alpha=FALSE)
ggsave("DA.pdf", p, width = 4.5, height = 4.5, units = "in")

Warning message:
“The dot-dot notation (`..x..`) was deprecated in ggplot2 3.4.0.
ℹ Please use `after_stat(x)` instead.”
Orientation inferred to be along y-axis; override with
`position_quasirandom(orientation = 'x')`
Orientation inferred to be along y-axis; override with
`position_quasirandom(orientation = 'x')`


In [106]:
ggsave("/data/hanxue/BCY_ATAC/joint_output/DA.svg",plot = p, device = "svg")

Saving 7 x 7 in image
Orientation inferred to be along y-axis; override with
`position_quasirandom(orientation = 'x')`
Orientation inferred to be along y-axis; override with
`position_quasirandom(orientation = 'x')`
